<a href="https://colab.research.google.com/github/nkwio/360Controller/blob/master/Absenteeism_Data_Import_NKW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notes on Code

**Author :**  Neil Walker
**Date :**     7th April 2020  
**Purpose :**  Training  
**Overview :** Opens up data file and preprocesses ready for use by an  
              Artifical Neural Net in order to predict levels of absenteeism on a  
              person by person basis.

Examine the ‘Absenteeism_data.csv’ carefully. Then, use the following as a guide to how you should prepare the data for further
analysis:
  
  Drop the ‘ID’ column  
  Split the reasons for absence into multiple dummy variables, and then group them in the following way:  
  Group 1: Columns 1 to 14  
  Group 2: Columns 15, 16, and 17  
  Group 3: Columns 18, 19, 20, and 21  
  Group 4: Columns 22 to 28  
    After you’ve done that, don’t forget to drop the ‘Reason for Absence’ column.  
    Extract the month value and the day of the week from the ‘Date’ column. Then, drop the ‘Date’ column as well.  
    Turn the data from the ‘Education’ column into binary data, by mapping the value of 0 to the values of 1, and the value of 1 to
the rest of the values found in this column.

## Start New Session

### Connect to Google Drive

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load a copy of raw data CSV file into memory



In [0]:
import pandas as pd
path = '/content/drive/Shared drives/GoogleColab/absenteeism/'
raw_csv_data = pd.read_csv(path+'Absenteeism_data.csv')
df = raw_csv_data.copy()
pd.options.display.max_columns = None
pd.options.display.max_rows = None

### Drop nominal variable

In [0]:
df = df.drop(['ID'], axis=1)

### Create dummy variables for "Reason for Absence"

In [229]:
reason_columns = pd.get_dummies(df['Reason for Absence'])
reason_columns['check'] = reason_columns.sum(axis=1)
if reason_columns['check'].unique() == 1: print('Categories are mutually exclusive')
reason_columns = reason_columns.drop('check', axis=1)
reason_columns = pd.get_dummies(df['Reason for Absence'], drop_first=True)

Categories are mutually exclusive


### Classify and Group the 'Reasons for Absence'

In [0]:
reason_type_1=reason_columns.loc[:,1:14].max(axis=1)
reason_type_2=reason_columns.loc[:,15:17].max(axis=1)
reason_type_3=reason_columns.loc[:,18:21].max(axis=1)
reason_type_4=reason_columns.loc[:,22:].max(axis=1)

In [0]:
df = pd.concat([df, reason_type_1, reason_type_2, reason_type_3, reason_type_4], axis=1)

In [0]:
column_names = (df.columns.values).tolist()
for i in range(1,5): column_names.pop()
for i in range(1,5): column_names.append('reason%s' % i)
df.columns = column_names

### Reorder Columns

In [0]:
column_names_reordered = ['reason1', 'reason2', 'reason3', 'reason4', 'Date', 'Transportation Expense', 'Distance to Work', 'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education', 'Children', 'Pets', 'Absenteeism Time in Hours']
df = df[column_names_reordered]

### Create dummy variable for "Age"

In [0]:
age_column = pd.get_dummies(df['Age'])
age_column['check'] = age_column.sum(axis=1)
if age_column['check'].unique() == 1: print('Categories are mutually exclusive')
age_column = age_column.drop('check', axis=1)
df = df.drop('Age', axis=1)
df_concatenated = pd.concat([df, age_column], axis=1)
df_concatenated.head()

In [0]:
column_names = ['reason1','reason2','reason3','reason4','Date', 'Transportation Expense',
       'Distance to Work', 'Daily Work Load Average', 'Body Mass Index',
       'Education', 'Children', 'Pets', 27,
       28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 43, 46, 47, 48,
       49, 50, 58, 'Absenteeism Time in Hours']
df_concatenated = df_concatenated[column_names]
df_concatenated.head()

In [0]:
df_reason_mod = df_concatenated.copy()


In [0]:
df_reason_mod['Date'] = pd.to_datetime(df_reason_mod['Date'], format='%d/%m/%Y')

In [0]:
df_working = df_reason_mod.copy()
list_months = []
for i in range(df_working.shape[0]):
  list_months.append(df_working['Date'][i].month)
df_working['Month Value'] = list_months
df = df_working.copy()

In [0]:
def date_to_weekday(date_value):
  return date_value.weekday()

df_working = df.copy()
df_working['Day of the Week'] = df_working['Date'].apply(date_to_weekday)
df_working = df_working.drop(['Date'],axis=1)
df = df_working.copy()

In [0]:
new_columns = ['reason1', 'reason2', 'reason3', 'reason4','Month Value',
               'Day of the Week', 'Transportation Expense', 'Distance to Work',
               'Daily Work Load Average', 'Body Mass Index', 'Education',
               'Children', 'Pets', 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38,
               39, 40, 41, 43, 46, 47, 48, 49, 50, 58,
               'Absenteeism Time in Hours'
               ]
df_working = df.copy()
df_working = df_working[new_columns]
df = df_working.copy()

### Re-categorise the Education Column

In [0]:
df_working = df.copy()
df_working['Education'] = df_working['Education'].map({1:0, 2:1, 3:1, 4:1})
df = df_working.copy()

## Final Checks & Export to CSV

In [0]:
df_preprocessed = df.copy()
df_preprocessed.head(10)

In [0]:
df_preprocessed.to_csv(path+'Absenteeism_preprocessed.csv', index=False)

## Close Down Session

### Flush data to Google Drive and Disconnect

In [0]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.
